In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.asymmetric.padding import OAEP, MGF1
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet

def generate_rsa_keys():
    # Generate private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )

    # Generate public key
    public_key = private_key.public_key()
    return private_key, public_key

def encrypt_message_with_public_key(message, public_key):
    encrypted_message = public_key.encrypt(
        message,
        OAEP(
            mgf=MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_message

def decrypt_message_with_private_key(encrypted_message, private_key):
    decrypted_message = private_key.decrypt(
        encrypted_message,
        OAEP(
            mgf=MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted_message

def symmetric_encrypt_message(message, key):
    f = Fernet(key)
    return f.encrypt(message)

def symmetric_decrypt_message(encrypted_message, key):
    f = Fernet(key)
    return f.decrypt(encrypted_message)

# Server Side
server_private_key, server_public_key = generate_rsa_keys()

# Simulate client side
# Client requests server's public key (simulated here by directly accessing it)
# Generate symmetric key for encryption
symmetric_key = Fernet.generate_key()

# Client encrypts the symmetric key using server's public key
encrypted_symmetric_key = encrypt_message_with_public_key(symmetric_key, server_public_key)

# Server decrypts the received symmetric key using its private key
decrypted_symmetric_key = decrypt_message_with_private_key(encrypted_symmetric_key, server_private_key)

# Test the encryption and decryption process with symmetric key
test_message = b"Hello, this is a secret message!"
encrypted_message = symmetric_encrypt_message(test_message, decrypted_symmetric_key)
decrypted_message = symmetric_decrypt_message(encrypted_message, decrypted_symmetric_key)

test_message, encrypted_message, decrypted_message


(b'Hello, this is a secret message!',
 b'gAAAAABljCc4_nlkWOJHYyWmojWRpo3E3UK9BZaqVzcgGiyJHFHYbYxzZpdGCVIAALjbL035uiqdgn2supb8WdhgpYt5FMmeop0UoCJBV8IE94_I5oFMf4n0bsd7_R58diozICt77tDi',
 b'Hello, this is a secret message!')